In [1]:
import json
import numpy as np
import os

In [2]:
from pinn import *
from nn import NN

torch.set_default_dtype(torch.float32)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
def serialize_savelist(keys: list, values : list, filename):
    dict = {k: v for k, v in zip(keys, values)}
    filename = f"{filename}.json"
    with open(filename, 'w') as file:
        json.dump(dict, file)

In [4]:
# Parameters model
rho = 0.5
mu = 79300 # MPa
lam = 100000 # MPa

Lx = 0.2
Ly = 0.2
T = 0.2

x_domain = np.array([0.0, Lx])/Lx
y_domain = np.array([0.0, Ly])/Lx
t_domain = np.array([0.0, T])/T

# Parameters NN
epochs = 12000
n_train = 30
layers = 2
dim_hidden = 40
lr = 0.02
om_INITIAL = 2
om_BOUNDARY = 0.4

# Eventually save changed $NN_2$ parameters to pass to other script

In [5]:
par = [Lx, T, n_train, layers, dim_hidden]
keys = ['x', 't', 'n', 'n_layers', 'n_neurons']

serialize_savelist(keys, par, 'par')

In [6]:
pinn = PINN(layers, dim_hidden, act=nn.Tanh()).to(device)

loss_fn = Loss(
    x_domain,
    y_domain,
    t_domain,
    n_train,
    return_adim(x_domain, t_domain, rho, mu, lam),
    initial_conditions,
    om_INITIAL,
    om_BOUNDARY
)

# Import weights from pretrained model

In [7]:
filename_model = get_last_modified_file('in_model')
pretrained_model_dict = torch.load(filename_model, map_location=torch.device(device))

pretrained_model = NN(layers, dim_hidden, 2, 1)
pretrained_model.load_state_dict(pretrained_model_dict)

<All keys matched successfully>

In [8]:
for i in np.arange(len(pinn.middle_layers)):
    pinn_layer = pinn.middle_layers[i]
    pretrained_layer = pretrained_model.middle_layers[i]
    pinn.middle_layers[i].weight.data.copy_(pretrained_model.middle_layers[i].weight)
    pinn.middle_layers[i].bias.data.copy_(pretrained_model.middle_layers[i].bias)

# Train new network with pretrained weights and biases in middle layers

In [9]:
pinn_trained, loss_values = train_model(
    pinn, loss_fn=loss_fn, learning_rate=lr, max_epochs=epochs)

Folder '2024-04-27' already exists.


Training:   0%|          | 0/12000 [00:00<?, ?it/s]

tensor([[ 5.4608e+10, -6.4805e+10, -3.6454e+10],
        [-1.3268e+11,  1.2824e+11,  8.1850e+10],
        [-1.5572e+11,  1.0803e+11,  7.0015e+10],
        [-3.8595e+10,  2.3176e+10,  1.4707e+10],
        [ 8.2128e+09, -1.1927e+10, -7.7690e+09],
        [ 2.3872e+11, -1.7897e+11, -1.2382e+11],
        [ 2.0958e+11, -1.7823e+11, -1.3813e+11],
        [ 3.2692e+10, -2.2688e+10, -1.5494e+10],
        [-1.4872e+11,  1.4883e+11,  9.7276e+10],
        [-1.0752e+10,  1.5177e+09,  1.0095e+09],
        [-1.1442e+10, -3.3084e+09, -2.3866e+09],
        [ 1.1265e+11, -9.3388e+10, -6.5246e+10],
        [ 4.9542e+10, -3.2531e+10, -1.0660e+10],
        [-3.8147e+10,  3.6561e+10,  2.7250e+10],
        [ 2.7454e+11, -2.2674e+11, -1.4976e+11],
        [-2.9544e+10,  3.6343e+10,  1.9756e+10],
        [ 2.3360e+11, -1.8331e+11, -1.1825e+11],
        [ 7.7561e+10, -5.8030e+10, -3.9280e+10],
        [-3.4599e+09,  4.0779e+09,  7.0377e+09],
        [-7.6545e+10,  5.4610e+10,  2.9522e+10],
        [-7.4135e+09

Loss: 4040360954966235615854592.0000:   0%|          | 1/12000 [00:01<6:12:17,  1.86s/it]

tensor([[ 4.1315e+10, -5.2014e+10, -2.3192e+10],
        [-9.9549e+10,  1.0063e+11,  5.0924e+10],
        [-1.1446e+11,  8.5179e+10,  4.1348e+10],
        [-2.9720e+10,  1.8134e+10,  1.0464e+10],
        [ 9.4855e+09, -8.6197e+09, -5.6231e+09],
        [ 1.8112e+11, -1.3790e+11, -7.5739e+10],
        [ 1.5658e+11, -1.3261e+11, -7.9141e+10],
        [ 2.5597e+10, -1.7764e+10, -1.0639e+10],
        [-1.1123e+11,  1.1702e+11,  6.0448e+10],
        [-9.9142e+09,  1.7333e+09,  2.7590e+09],
        [-8.7287e+09, -1.0174e+09,  6.0196e+08],
        [ 8.1972e+10, -6.9671e+10, -3.8070e+10],
        [ 3.8486e+10, -2.6855e+10, -9.1604e+09],
        [-2.9282e+10,  2.9059e+10,  1.7857e+10],
        [ 2.0746e+11, -1.7524e+11, -9.0795e+10],
        [-2.1527e+10,  2.8906e+10,  1.1997e+10],
        [ 1.7525e+11, -1.4054e+11, -7.0339e+10],
        [ 6.9094e+10, -4.9250e+10, -3.1715e+10],
        [-3.1563e+09,  3.5466e+09,  5.2912e+09],
        [-5.6717e+10,  3.6299e+10,  1.4056e+10],
        [-4.9858e+09

Loss: 40401458990768294199296.0000:   0%|          | 2/12000 [00:03<5:26:46,  1.63s/it]  

tensor([[ 3.4992e+10, -4.4670e+10, -1.7065e+10],
        [-8.3555e+10,  8.4952e+10,  3.7154e+10],
        [-9.3820e+10,  7.1637e+10,  2.8564e+10],
        [-2.5476e+10,  1.5450e+10,  8.4478e+09],
        [ 9.8643e+09, -6.9107e+09, -4.6744e+09],
        [ 1.5240e+11, -1.1405e+11, -5.3848e+10],
        [ 1.3053e+11, -1.0907e+11, -5.5201e+10],
        [ 2.2410e+10, -1.5247e+10, -8.5296e+09],
        [-9.2832e+10,  9.8684e+10,  4.3922e+10],
        [-9.6504e+09,  1.6163e+09,  3.1399e+09],
        [-7.7002e+09, -3.9414e+08,  1.3751e+09],
        [ 6.7315e+10, -5.6900e+10, -2.6682e+10],
        [ 3.3761e+10, -2.3386e+10, -7.8846e+09],
        [-2.5300e+10,  2.4858e+10,  1.3597e+10],
        [ 1.7433e+11, -1.4615e+11, -6.4504e+10],
        [-1.8051e+10,  2.4696e+10,  8.6494e+09],
        [ 1.4639e+11, -1.1623e+11, -4.8964e+10],
        [ 6.4693e+10, -4.4024e+10, -2.7599e+10],
        [-3.2538e+09,  3.3637e+09,  4.5263e+09],
        [-4.5683e+10,  2.7529e+10,  7.7448e+09],
        [-3.7703e+09

Loss: 404000541491595313152.0000:   0%|          | 3/12000 [00:04<4:43:25,  1.42s/it]  

KeyboardInterrupt: 

In [ ]:
current_time = get_current_time(fmt="%H:%M")

folder = 'model'
model_name = f'{lr}_{epochs}_{dim_hidden}_{current_time}.pth'
model_path = os.path.join(folder, model_name)
os.makedirs(folder, exist_ok=True)

torch.save(pinn_trained.state_dict(), model_path)

In [ ]:
from plots import plot_initial_conditions, plot_solution

x, y, _ = get_initial_points(x_domain, y_domain, t_domain, n_train)
t_value = 0.0
t = torch.full_like(x, t_value)
x = x.to(device)
y = y.to(device)
t = t.to(device)
z = f(pinn, x ,y, t)
ux_0, uy_0 = initial_conditions(x, y, Lx, i = 1)
z_0 = torch.cat((ux_0, uy_0), dim=1)

plot_initial_conditions(z_0, y, x, 'Initial conditions - analytical', n_train, from_pinn = 0)
plot_initial_conditions(z, y, x, 'Initial conditions - NN', n_train)

In [ ]:
from IPython.display import HTML

x, y, t = get_interior_points(x_domain, y_domain, t_domain, n_train)
animation = plot_solution(pinn, x, y, t, n_train)
HTML(animation.to_jshtml())

# To be added
- ~separate loss in more bars to see how the various loss term come to zero~
- see if some quadrature rule has been implemented
- scheme of weights initialization in order to automatically satisfy initial conditions
- plots (in progress)
- NN operators (to generalize results)
- try to implement function that allows that satisfy initial conditions?

# To be fully understood
- reshape when plotting (also different number of points allowed?)
- ~not consistent initial conditions~ (between $u_x$ and $u_y$)?